# Essential Gene Review

In [195]:
# Set up PATH for imports
import sys
if ".." not in sys.path:
    sys.path.append("..")
    
import cobra
import pandas as pd
from cobra.io import read_sbml_model

model = read_sbml_model("../model/Rpom_05.xml")
model

Name,Rpom_05
Memory address,12940a0d0
Number of metabolites,1797
Number of reactions,1796
Number of genes,968
Number of groups,0
Objective expression,1.0*Rpom_hwa_biomass - 1.0*Rpom_hwa_biomass_reverse_5ec2f
Compartments,"c, p, e"


In [196]:
def check_essential_rxn(rxn_id):
    essential_rxn = model.reactions.get_by_id(rxn_id)
    with model:
        model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)

        sol = model.optimize()
        print(f"Growth rate with " + rxn_id + f": {sol.objective_value:.4f} 1/hr")
        baseline_flux = sol.fluxes[essential_rxn.id]
        print(f"Flux through " + rxn_id + f": {baseline_flux}\n")

        essential_rxn.knock_out()

        sol = model.optimize()
        print(f"Growth rate without " + rxn_id + f": {sol.objective_value:.4f} 1/hr")
    if sol.objective_value == 0:
        return True, baseline_flux
    else:
        return False, baseline_flux
    
def add_source(rxn_id, to_add_id): 
    essential_rxn = model.reactions.get_by_id(rxn_id)
    with model:
        model.reactions.get_by_id('EX_glc').bounds = (-5.44, 0)

        # Baseline growth rate is 0, with tptase knocked out
        sol = model.optimize()
        is_essential, baseline_flux = check_essential_rxn(rxn_id)

        essential_rxn.knock_out()
        # Does adding free to_add fix things?
        source = model.add_boundary(
            model.metabolites.get_by_id(to_add_id),
            type="sink",
            reaction_id=to_add_id + "_source",
            lb=-1 * baseline_flux,  # Set to exactly the lost flux from above!
            ub=0
        )
        print(f"New reaction: {source.id}\n\t{source.reaction}\t\t{source.bounds}")

        sol = model.optimize()
        print(f"Growth rate with new reaction: {sol.objective_value:.2f} 1/hr")
        print()
        
def add_sink(rxn_id, to_sink_id): 
    essential_rxn = model.reactions.get_by_id(rxn_id)
    with model:
        model.reactions.get_by_id('EX_glc').bounds = (-5.44, 0)

        # Baseline growth rate is 0, with tptase knocked out
        sol = model.optimize()
        (is_essential, baseline_flux) = check_essential_rxn(rxn_id)

        essential_rxn.knock_out()
        # Does adding free to_add fix things?
        sink = model.add_boundary(
            model.metabolites.get_by_id(to_sink_id),
            type="sink",
            reaction_id=to_sink_id + "_sink",
            ub=baseline_flux,  # Set to exactly the lost flux from above!
            lb=0
        )
        print(f"New reaction: {sink.id}\n\t{sink.reaction}\t\t{sink.bounds}")

        sol = model.optimize()
        print(f"Growth rate with new reaction: {sol.objective_value:.2f} 1/hr")
        print()

def get_metabolite_reactions(met_id): 
    met = model.metabolites.get_by_id(met_id)

    rxns = list(met.reactions)          # list of reactions
    rxn_ids = [r.id for r in rxns]      # just IDs
    consuming = [r for r in met.reactions if r.metabolites[met] < 0]
    producing = [r for r in met.reactions if r.metabolites[met] > 0]

    print(f"Consuming reactions:")
    for rxn in consuming:
        print(f"\t{rxn.id}: {rxn.reaction}")

    print(f"Producing reactions:")
    for rxn in producing:
        print(f"\t{rxn.id}: {rxn.reaction}")
    print()
    return consuming, producing
        
        
def add_source_and_sink(rxn_id, to_add_id, to_sink_id): 
    essential_rxn = model.reactions.get_by_id(rxn_id)
    with model:
        model.reactions.get_by_id('EX_glc').bounds = (-5.44, 0)

        # Baseline growth rate is 0, with tptase knocked out
        sol = model.optimize()
        is_essential, baseline_flux = check_essential_rxn(rxn_id)

        essential_rxn.knock_out()
        # Does adding free to_add fix things?
        source = model.add_boundary(
            model.metabolites.get_by_id(to_add_id),
            type="sink",
            reaction_id=to_add_id + "_source",
            lb=-1 * baseline_flux,  # Set to exactly the lost flux from above!
            ub=0
        )
        print(f"New reaction: {source.id}\n\t{source.reaction}\t\t{source.bounds}")
        
        sink = model.add_boundary(
            model.metabolites.get_by_id(to_sink_id),
            type="sink",
            reaction_id=to_sink_id + "_sink",
            ub=baseline_flux,  # Set to exactly the lost flux from above!
            lb=0
        )
        print(f"New reaction: {sink.id}\n\t{sink.reaction}\t\t{sink.bounds}")

        sol = model.optimize()
        print(f"Growth rate with new reaction: {sol.objective_value:.2f} 1/hr")
        print()
        
def add_many_source_and_sink(rxn_id, to_add_ids, to_sink_ids): 
    essential_rxn = model.reactions.get_by_id(rxn_id)
    with model:
        model.reactions.get_by_id('EX_glc').bounds = (-5.44, 0)

        # Baseline growth rate is 0, with tptase knocked out
        sol = model.optimize()
        is_essential, baseline_flux = check_essential_rxn(rxn_id)

        essential_rxn.knock_out()
        # Does adding free to_add fix things?
        for to_add_id in to_add_ids:
            source = model.add_boundary(
                model.metabolites.get_by_id(to_add_id),
                type="sink",
                reaction_id=to_add_id + "_source",
                lb=-1 * baseline_flux,  # Set to exactly the lost flux from above!
                ub=0
            )
            print(f"New reaction: {source.id}\n\t{source.reaction}\t\t{source.bounds}")
        
        for to_sink_id in to_sink_ids:
            sink = model.add_boundary(
                model.metabolites.get_by_id(to_sink_id),
                type="sink",
                reaction_id=to_sink_id + "_sink",
                ub=baseline_flux,  # Set to exactly the lost flux from above!
                lb=0
            )
            print(f"New reaction: {sink.id}\n\t{sink.reaction}\t\t{sink.bounds}")

        sol = model.optimize()
        print(f"Growth rate with new reaction: {sol.objective_value:.2f} 1/hr")
        print()

def print_reaction_info(rxn_id):
    rxn = model.reactions.get_by_id(rxn_id)
    print(f"{rxn.id}:\n\t{rxn.reaction}\n")
    print(f"Bounds: {rxn.bounds}")
    print(f"Reversibility: {rxn.reversibility}")
    print(f"Genes: {rxn.genes}")
    print(f"Subsystem: {rxn.subsystem}")

## 1. ASNSYNB-RXN

In [197]:
rxn_id = "ASNSYNB-RXN"
essential_rxn = model.reactions.get_by_id(rxn_id)
print(f"{essential_rxn.id}:\n\t{essential_rxn.reaction}\n")

test_essential, baseline_flux = check_essential_rxn(rxn_id)

ASNSYNB-RXN:
	ATP[c] + GLN[c] + L-ASPARTATE[c] + WATER[c] --> AMP[c] + ASN[c] + GLT[c] + PPI[c] + PROTON[c]

Growth rate with ASNSYNB-RXN: 0.5680 1/hr
Flux through ASNSYNB-RXN: 0.11376672577012338

Growth rate without ASNSYNB-RXN: 0.0000 1/hr


Interestingly, ASNSYNB-RXN is not in biocyc or kegg. This means it is likely not in R Pom.

Why might ASNSYNB-RXN be essential?

1. Maybe we need to generate ASN?
2. Maybe we need to eliminate L-Aspartate?



In [198]:
add_source("ASNSYNB-RXN", "ASN[c]")

Growth rate with ASNSYNB-RXN: 0.5680 1/hr
Flux through ASNSYNB-RXN: 0.11376672577012077

Growth rate without ASNSYNB-RXN: 0.0000 1/hr
New reaction: ASN[c]_source
	ASN[c] <-- 		(np.float64(-0.11376672577012077), 0)
Growth rate with new reaction: 0.57 1/hr



In [199]:
add_sink("ASNSYNB-RXN", "L-ASPARTATE[c]")

Growth rate with ASNSYNB-RXN: 0.5680 1/hr
Flux through ASNSYNB-RXN: 0.1137667257701208

Growth rate without ASNSYNB-RXN: 0.0000 1/hr
New reaction: L-ASPARTATE[c]_sink
	L-ASPARTATE[c] --> 		(0, np.float64(0.1137667257701208))
Growth rate with new reaction: 0.00 1/hr



It seems that hypothesis 1 holds.

There are 2 possibilities here:

1. Either ASN[c] is not present in R Pom, and we need to eliminate a bunch of downstream reactions...
2. R Pom has an alternative means of producing ASN[c]

Biocyc indicates ASN (L-asparagine) is indeed in R Pom, so H2 is likely true here. 

In [200]:
met_id = "ASN[c]"  
consuming, producing = get_metabolite_reactions(met_id)

Consuming reactions:
	RXN-20896: ASN[c] <=> CPD-3633[c]
	ASPARAGHYD-RXN: ASN[c] + WATER[c] --> AMMONIUM[c] + L-ASPARTATE[c]
	PROTEIN-RXN: 0.437725018 ARG[c] + 0.356722982 ASN[c] + 33.1963969 ATP[c] + 0.135523805 CYS[c] + 0.389434859 GLN[c] + 0.389434859 GLT[c] + 0.906606601 GLY[c] + 0.140197141 HIS[c] + 0.429936617 ILE[c] + 0.760177851 L-ALPHA-ALANINE[c] + 0.356722982 L-ASPARTATE[c] + 0.666714077 LEU[c] + 0.507823589 LYS[c] + 0.227430786 MET[c] + 0.274162674 PHE[c] + 0.32712617 PRO[c] + 0.319337768 SER[c] + 0.375416329 THR[c] + 0.084118581 TRP[c] + 0.204064103 TYR[c] + 0.626212319 VAL[c] --> 33.1963969 ADP[c] + PROTEIN[c] + 33.1963969 PROTON[c] + 33.1963969 Pi[c]
Producing reactions:
	ASNSYNB-RXN: ATP[c] + GLN[c] + L-ASPARTATE[c] + WATER[c] --> AMP[c] + ASN[c] + GLT[c] + PPI[c] + PROTON[c]
	RXN1RHL-12: ASN[p] + ATP[c] + WATER[c] --> ADP[c] + ASN[c] + PROTON[c] + Pi[c]



- **Reaction RXN1RHL-12 in biocyc**
- ASPARAGHYD-RXN not in biocyc
- RXN-20896 not in biocyc
- PROTEIN-RXN resembles a pseudo-reaction for protein synthesis (translation)

In [201]:
consuming, producing = get_metabolite_reactions("ASN[p]")

Consuming reactions:
	RXN1RHL-12: ASN[p] + ATP[c] + WATER[c] --> ADP[c] + ASN[c] + PROTON[c] + Pi[c]
Producing reactions:
	ASNtex: ASN[e] <=> ASN[p]



In [202]:
consuming, producing = get_metabolite_reactions("PROTEIN[c]")

Consuming reactions:
	Rpom_hwa_biomass: 31.36179486 ATP[c] + 0.001524574 COFACTORS[c] + 0.05136379 DNA[c] + 0.009825009 IONS[c] + 0.095950926 LIPID[c] + 0.026355529 MUREIN[c] + 0.000447 NADP[c] + 0.001831 NAD[c] + 0.091372857 PHB-STORAGE[c] + 0.561521914 PROTEIN[c] + 0.093597842 RNA[c] + 48.601527 WATER[c] --> 31.36179573 ADP[c] + 31.36179573 PROTON[c] + 31.35745773 Pi[c]
Producing reactions:
	PROTEIN-RXN: 0.437725018 ARG[c] + 0.356722982 ASN[c] + 33.1963969 ATP[c] + 0.135523805 CYS[c] + 0.389434859 GLN[c] + 0.389434859 GLT[c] + 0.906606601 GLY[c] + 0.140197141 HIS[c] + 0.429936617 ILE[c] + 0.760177851 L-ALPHA-ALANINE[c] + 0.356722982 L-ASPARTATE[c] + 0.666714077 LEU[c] + 0.507823589 LYS[c] + 0.227430786 MET[c] + 0.274162674 PHE[c] + 0.32712617 PRO[c] + 0.319337768 SER[c] + 0.375416329 THR[c] + 0.084118581 TRP[c] + 0.204064103 TYR[c] + 0.626212319 VAL[c] --> 33.1963969 ADP[c] + PROTEIN[c] + 33.1963969 PROTON[c] + 33.1963969 Pi[c]



Either ASN is not needed (need to modify protein synthesis reaction)

OR 

We need to give the cell ASN[e] at the start

The second option seems more plausible.

https://www.nature.com/articles/ismej20166 

States that R. pomeroyi can synthesize all amino acids except asparagine and possibly histidine.

But Wait!

https://www.pnas.org/doi/10.1073/pnas.2217200120

Shows that R. Pom can grow without asparagine in media...

So there must be a means to explain what is going on...

Likely has to do with Protein Reaction

In [203]:
check_essential_rxn("PROTEIN-RXN")

Growth rate with PROTEIN-RXN: 0.5680 1/hr
Flux through PROTEIN-RXN: 0.31892177266594507

Growth rate without PROTEIN-RXN: 0.0000 1/hr


(True, np.float64(0.31892177266594507))

What is likely going on is that either:

1. When no external asparagine is present, the cell uses reaction 6.1.1.23, using L-aspartate at asn tRNA
    -   l-asparaginyl tRNA[ASN] can  also be formed by reaction RXN-20081, 6.3.5.6-RXN, using L-aspartate-tRNA[ASN].
2. Otherwise the cell imports L-aspartate normally. 

Not in any database! Essential for translation reaction. However, translation reaction neglects tRNA. While R Pom lacks traditional ASN synthesis pathways (https://www.nature.com/articles/ismej20166 ), it is able to grow in ASN-free media (https://www.pnas.org/doi/10.1073/pnas.2217200120). This is likely because L-asparaginyl-tRNA[ASN] can be synthesized independetly of ASN, from L-Aspartate: reaction 6.1.1.23, followed by RXN-20081, 6.3.5.6-RXN. Proposed Fix: either we cut this reaction and add ASN as an essential metabolite, or we keep the reaction (which seems better to me) as a pseudo-capture of the tRNA behavior (note tRNA behavior also uses amine or ATP, so perhaps we could modify this reaction into 2 separate ones which incorporate those).

## 2. AACPS4

In [204]:
rxn_id = "AACPS4"
essential_rxn = model.reactions.get_by_id(rxn_id)
print(f"{essential_rxn.id}:\n\t{essential_rxn.reaction}\n")

test_essential, baseline_flux = check_essential_rxn(rxn_id)

AACPS4:
	ACP[c] + ATP[c] + hdcea[c] <=> AMP[c] + PPI[c] + hdeACP[c]

Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919128167

Growth rate without AACPS4: 0.0000 1/hr


Interestingly, AACPS4 is not in biocyc or kegg. This means it is likely not in R Pom.

Why might AACPS4 be essential?

1. Maybe we need to generate hdeACP?
2. Maybe we need to eliminate hdcea?

In [205]:
add_source("AACPS4", "hdeACP[c]")
print("_____")
add_source("AACPS4", "PPI[c]")
print("_____")
add_source("AACPS4", "AMP[c]")
print("_____")

Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.0854370291910244

Growth rate without AACPS4: 0.0000 1/hr
New reaction: hdeACP[c]_source
	hdeACP[c] <-- 		(np.float64(-0.0854370291910244), 0)
Growth rate with new reaction: 0.00 1/hr

_____
Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102389

Growth rate without AACPS4: 0.0000 1/hr
New reaction: PPI[c]_source
	PPI[c] <-- 		(np.float64(-0.08543702919102389), 0)
Growth rate with new reaction: 0.00 1/hr

_____
Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102378

Growth rate without AACPS4: 0.0000 1/hr
New reaction: AMP[c]_source
	AMP[c] <-- 		(np.float64(-0.08543702919102378), 0)
Growth rate with new reaction: 0.00 1/hr

_____


In [206]:
add_sink("AACPS4", "hdcea[c]")
print("_____")
add_sink("AACPS4", "ACP[c]")
print("_____")
add_sink("AACPS4", "ATP[c]")
print("_____")

Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102378

Growth rate without AACPS4: 0.0000 1/hr
New reaction: hdcea[c]_sink
	hdcea[c] --> 		(0, np.float64(0.08543702919102378))
Growth rate with new reaction: 0.00 1/hr

_____
Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102378

Growth rate without AACPS4: 0.0000 1/hr
New reaction: ACP[c]_sink
	ACP[c] --> 		(0, np.float64(0.08543702919102378))
Growth rate with new reaction: 0.00 1/hr

_____
Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102378

Growth rate without AACPS4: 0.0000 1/hr
New reaction: ATP[c]_sink
	ATP[c] --> 		(0, np.float64(0.08543702919102378))
Growth rate with new reaction: 0.00 1/hr

_____


In [207]:
add_source_and_sink("AACPS4", "hdeACP[c]", "hdcea[c]")


Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102378

Growth rate without AACPS4: 0.0000 1/hr
New reaction: hdeACP[c]_source
	hdeACP[c] <-- 		(np.float64(-0.08543702919102378), 0)
New reaction: hdcea[c]_sink
	hdcea[c] --> 		(0, np.float64(0.08543702919102378))
Growth rate with new reaction: 0.00 1/hr



Confusing... Let's brute force to figure out which are necessary!

In [208]:
add_many_source_and_sink("AACPS4", ["hdeACP[c]"], ["ACP[c]"])

Growth rate with AACPS4: 0.5680 1/hr
Flux through AACPS4: 0.08543702919102303

Growth rate without AACPS4: 0.0000 1/hr
New reaction: hdeACP[c]_source
	hdeACP[c] <-- 		(np.float64(-0.08543702919102303), 0)
New reaction: ACP[c]_sink
	ACP[c] --> 		(0, np.float64(0.08543702919102303))
Growth rate with new reaction: 0.57 1/hr



So it seems like we need to generate hdeACP[c] and destroy ACP[c] somehow!



In [209]:
consuming, producing = get_metabolite_reactions("hdcea[c]")

Consuming reactions:
	AACPS4: ACP[c] + ATP[c] + hdcea[c] <=> AMP[c] + PPI[c] + hdeACP[c]
Producing reactions:
	FACOAE161: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]



In [210]:
consuming, producing = get_metabolite_reactions("hdeACP[c]")

Consuming reactions:
	1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN: ACYL-SN-GLYCEROL-3P[c] + hdeACP[c] --> ACP[c] + L-PHOSPHATIDATE[c]
	RXN-10462: GLYCEROL-3P[c] + hdeACP[c] --> ACP[c] + ACYL-SN-GLYCEROL-3P[c]
Producing reactions:
	AACPS4: ACP[c] + ATP[c] + hdcea[c] <=> AMP[c] + PPI[c] + hdeACP[c]



In [211]:
consuming2, producing2 = get_metabolite_reactions("ACP[c]")

Consuming reactions:
	MALONYL-COA-ACP-TRANSACYL-RXN: ACP[c] + MALONYL-COA[c] --> CO-A[c] + MALONYL-ACP[c]
	AACPS4: ACP[c] + ATP[c] + hdcea[c] <=> AMP[c] + PPI[c] + hdeACP[c]
Producing reactions:
	RXN-9523: Hexanoyl-ACPs[c] + MALONYL-ACP[c] + PROTON[c] --> 3-Oxo-octanoyl-ACPs[c] + ACP[c] + CARBON-DIOXIDE[c]
	RXN-9548: Stearoyl-ACPs[c] + WATER[c] --> ACP[c] + PROTON[c] + STEARIC_ACID[c]
	RXN-10462: GLYCEROL-3P[c] + hdeACP[c] --> ACP[c] + ACYL-SN-GLYCEROL-3P[c]
	PALMITOTRANS-RXN: KDO2-LIPID-IVA[c] + Palmitoleoyl-ACPs[c] --> ACP[c] + KDO2-PALMITOLEOYL-LIPID-IVA[c]
	RXN-10025: Octanoyl-ACPs[c] + S-ADENOSYLMETHIONINE[c] <=> 5-METHYLTHIOADENOSINE[c] + ACP[c] + CPD-10785[c] + PROTON[c]
	RXN-10022: 3-oxo-hexanoyl-ACPs[c] + S-ADENOSYLMETHIONINE[c] <=> 5-METHYLTHIOADENOSINE[c] + ACP[c] + CPD-10780[c] + PROTON[c]
	UDPNACETYLGLUCOSAMACYLTRANS-RXN: R-3-hydroxymyristoyl-ACPs[c] + UDP-N-ACETYL-D-GLUCOSAMINE[c] --> ACP[c] + UDP-OHMYR-ACETYLGLUCOSAMINE[c]
	RXN-9535: Dodecanoyl-ACPs[c] + MALONYL-ACP[c] +

In [212]:
model.metabolites.get_by_id("ACP[c]")

Metabolite identifier,ACP[c]
Name,a soluble [acyl-carrier protein]
Memory address,0x12945ac40
Formula,XH
Compartment,c
In 32 reaction(s),"RXN-9523, RXN-9548, MALONYL-COA-ACP-TRANSACYL-RXN, RXN-10462, PALMITOTRANS-RXN, RXN-10025, RXN-10022, UDPNACETYLGLUCOSAMACYLTRANS-RXN, RXN-9535, RXN3O-1803, MYRISTOYLACYLTRAN-RXN, RXN-10023,..."


In [213]:
model.metabolites.get_by_id("hdeACP[c]").name

'Cis-hexadec-9-enoyl-[acyl-carrier protein] (n-C16:1)'

Maybe necessary for RXN-10462

In [214]:
print_reaction_info("RXN-10462")
print("================")
test_essential, baseline_flux = check_essential_rxn("RXN-10462")

RXN-10462:
	GLYCEROL-3P[c] + hdeACP[c] --> ACP[c] + ACYL-SN-GLYCEROL-3P[c]

Bounds: (0.0, 1000.0)
Reversibility: False
Genes: frozenset({<Gene SPO0369 at 0x12916d8b0>})
Subsystem: 
Growth rate with RXN-10462: 0.5680 1/hr
Flux through RXN-10462: 0.04271851459543866

Growth rate without RXN-10462: 0.0000 1/hr


In [215]:
add_source("RXN-10462", "ACYL-SN-GLYCEROL-3P[c]")

Growth rate with RXN-10462: 0.5680 1/hr
Flux through RXN-10462: 0.04271851459551131

Growth rate without RXN-10462: 0.0000 1/hr
New reaction: ACYL-SN-GLYCEROL-3P[c]_source
	ACYL-SN-GLYCEROL-3P[c] <-- 		(np.float64(-0.04271851459551131), 0)
Growth rate with new reaction: 0.57 1/hr



In [ ]:
get_metabolite_reactions("ACYL-SN-GLYCEROL-3P[c]")

In [216]:
get_metabolite_reactions("ACYL-SN-GLYCEROL-3P[c]")

Consuming reactions:
	1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN: ACYL-SN-GLYCEROL-3P[c] + hdeACP[c] --> ACP[c] + L-PHOSPHATIDATE[c]
Producing reactions:
	RXN-10462: GLYCEROL-3P[c] + hdeACP[c] --> ACP[c] + ACYL-SN-GLYCEROL-3P[c]



([<Reaction 1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN at 0x12839ad90>],
 [<Reaction RXN-10462 at 0x13c03c9d0>])

Suspect this has something to do with RXN 10462 taking acyl-[acyl-carrier protein] as input, not hdeACP.

10462 in database:
acyl-[acyl-carrier protein] + sn-glycerol 3-phosphate → a 2-lysophosphatidate + an acyl-carrier protein

It could also be the case that there needs to be a separate hdeACP synthesis pathway or an alternative means of producing **ACYL-SN-GLYCEROL-3P[c]** which is the critical component from RXN-10462.




What if we remove RXN-10462, AACPS4?

- For hdcea use defined reaction 6.2.1.3:

a long-chain fatty acid + coenzyme A + ATP → a long-chain fatty acyl-CoA + AMP + diphosphate

We can define the reaction

**hdcea[c] + CO-A[c] + ATP[c] -> hdceCOA[c] + AMP[c] + PPI[c]**

- Next, we can replace RXN-10462 with reaction 2.3.1.15:

an acyl-CoA + sn-glycerol 3-phosphate → a 2-lysophosphatidate + coenzyme A

or 

**GLYCEROL-3P[c] + hdeCOA[c] --> CO-A[c] + ACYL-SN-GLYCEROL-3P[c]**

- Lastly, we can replace 1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN with RXN 2.3.1.51:

an acyl-CoA + a 2-lysophosphatidate → a phosphatidate + coenzyme A

or 

**ACYL-SN-GLYCEROL-3P[c] + hdceCOA[c] -> L-PHOSPHATIDATE[c] + CO-A[c]**



## 3. FACOAE161

In [217]:
rxn_id = "FACOAE161"
essential_rxn = model.reactions.get_by_id(rxn_id)
print(f"{essential_rxn.id}:\n\t{essential_rxn.reaction}\n")

test_essential, baseline_flux = check_essential_rxn(rxn_id)

FACOAE161:
	POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]

Growth rate with FACOAE161: 0.5680 1/hr
Flux through FACOAE161: 0.08543702919102425

Growth rate without FACOAE161: 0.0000 1/hr


In [222]:
add_source("FACOAE161", "hdcea[c]")
#add_sink("FACOAE161", "POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c]")
get_metabolite_reactions("hdcea[c]")


Growth rate with FACOAE161: 0.5680 1/hr
Flux through FACOAE161: 0.08543702919102268

Growth rate without FACOAE161: 0.0000 1/hr
New reaction: hdcea[c]_source
	hdcea[c] <-- 		(np.float64(-0.08543702919102268), 0)
Growth rate with new reaction: 0.57 1/hr

Consuming reactions:
	AACPS4: ACP[c] + ATP[c] + hdcea[c] <=> AMP[c] + PPI[c] + hdeACP[c]
Producing reactions:
	FACOAE161: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]



([<Reaction AACPS4 at 0x129331eb0>], [<Reaction FACOAE161 at 0x1296b5f40>])

In [250]:
get_metabolite_reactions("POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c]")
get_metabolite_reactions("hdcea[c]")

Consuming reactions:
	ENOYL-COA-DELTA-ISOM-RXN-POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24//CPD0-2117.53.: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] <=> CPD0-2117[c]
	FACOAE161: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]
Producing reactions:

Consuming reactions:
	AACPS4: ACP[c] + ATP[c] + hdcea[c] <=> AMP[c] + PPI[c] + hdeACP[c]
Producing reactions:
	FACOAE161: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]



([<Reaction AACPS4 at 0x129331eb0>], [<Reaction FACOAE161 at 0x1296b5f40>])

"Not in database. Required to form hdcea[c]. Reactions known to produce this compound which we could replace FACOAE161 with:

1. Reaction 3.1.1.77:

a 3-(acyloxy)acyl group of bacterial toxin + H2O → a 3-hydroxyacyl group of bacterial toxin + a fatty acid + H+

2. Reaction 1.2.1.4:

an aldehyde + NADP+ + H2O → a carboxylate + NADPH + 2 H+

Here we would have some kind of:

**hexadecenal[c] + NADP+[c] + H2O[c] -> hdcea[c] + NADPH[c] + 2H+[c]**

I cannot find hexadecenal in the model or database however...

______________

http://bigg.ucsd.edu/universal/reactions/FACOAE161

suggets that FACOAE161 is actually a reaction which produces and derives from hdcoa_c. 

Ie hdCOA[c] is equivalent to POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c].

This is not the case at the molecular formula level, so maybe we need to redefine a new hdCOA[c] species.

We can define the reaction

**hdcea[c] + CO-A[c] + ATP[c] -> POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + AMP[c] + PPI[c]**

- Next, we can replace RXN-10462 with reaction 2.3.1.15:

an acyl-CoA + sn-glycerol 3-phosphate → a 2-lysophosphatidate + coenzyme A

or 

**GLYCEROL-3P[c] + POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] --> CO-A[c] + ACYL-SN-GLYCEROL-3P[c]** (1)

- Lastly, we can replace 1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN with RXN 2.3.1.51:

an acyl-CoA + a 2-lysophosphatidate → a phosphatidate + coenzyme A

or 

**ACYL-SN-GLYCEROL-3P[c] + POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] -> L-PHOSPHATIDATE[c] + CO-A[c]** (2)

What if we just keep (1) and (2)? And knockout both 10462 and FACOAE161?

However, maybe we need to remove POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c], and replace it with hdCOA separately. 



In [249]:
m = model.metabolites.get_by_id("POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c]")
print(m.id, m.name, m.formula)
for r in m.reactions:
    print(r.id, r.reaction)


POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24 C37H60N7O17P3S
ENOYL-COA-DELTA-ISOM-RXN-POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24//CPD0-2117.53. POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] <=> CPD0-2117[c]
FACOAE161 POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]


In [239]:
m = model.metabolites.get_by_id("CPD0-2117[c]")  # adjust formatting if needed
print(m.id, m.name, m.formula)

CPD0-2117[c] (2E)-hexadecenoyl-CoA C37H60N7O17P3S


In [240]:
m = model.metabolites.get_by_id("POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c]")  # adjust formatting if needed
print(m.id, m.name, m.formula)

POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24 C37H60N7O17P3S


In [242]:
m = model.metabolites.get_by_id("hdcea[c]")  # adjust formatting if needed
print(m.id, m.name, m.formula)

hdcea[c] palmitoleate C16H29O2


In [244]:
m = model.metabolites.get_by_id("L-PHOSPHATIDATE[c]")
print(m.id, m.name, m.formula)

L-PHOSPHATIDATE[c] a 1,2-diacyl-sn-glycerol 3-phosphate C35H63O8P


In [248]:
m = model.metabolites.get_by_id("ACYL-SN-GLYCEROL-3P[c]")
print(m.id, m.name, m.formula)

m = model.metabolites.get_by_id("GLYCEROL-3P[c]")
print(m.id, m.name, m.formula)

ACYL-SN-GLYCEROL-3P[c] a 1-acyl-sn-glycerol 3-phosphate C19H35O7P
GLYCEROL-3P[c] sn-glycerol 3-phosphate C3H7O6P


In [230]:
with model:
    model.reactions.get_by_id("FACOAE161").knock_out()
    model.reactions.get_by_id("RXN-10462").knock_out()
    
    # define relevant metabolites
    g3p = model.metabolites.get_by_id("GLYCEROL-3P[c]")
    hdcoa = model.metabolites.get_by_id("POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c]")
    coa = model.metabolites.get_by_id("CO-A[c]")
    acyl_sn_glycerol_3p = model.metabolites.get_by_id("ACYL-SN-GLYCEROL-3P[c]")
    l_phosphatidate = model.metabolites.get_by_id("L-PHOSPHATIDATE[c]")
    
    # define reaction 2.3.1.15
    rxn1 = cobra.Reaction("RXN_2.3.1.15")
    rxn1.add_metabolites({g3p: -1, hdcoa: -1, coa: 1, acyl_sn_glycerol_3p: 1})
    model.add_reactions([rxn1])
    
    # define reaction 2.3.1.51
    rxn2 = cobra.Reaction("RXN_2.3.1.51")
    rxn2.add_metabolites({acyl_sn_glycerol_3p: -1, hdcoa: -1, l_phosphatidate: 1, coa: 1})
    model.add_reactions([rxn2])
    
    # test model
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    sol = model.optimize()
    print(f"Growth rate with {sol.objective_value:.4f} 1/hr")
    


Growth rate with 0.5687 1/hr


## 4. LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.

In [231]:
rxn_id = "LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56."
essential_rxn = model.reactions.get_by_id(rxn_id)
print(f"{essential_rxn.id}:\n\t{essential_rxn.reaction}\n")

test_essential, baseline_flux = check_essential_rxn(rxn_id)



LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.:
	ETF-Oxidized[c] + PALMITYL-COA[c] + PROTON[c] <=> CPD0-2117[c] + ETF-Reduced[c]

Growth rate with LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.5680 1/hr
Flux through LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.08543702919102396

Growth rate without LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.0000 1/hr


In [238]:
# brute forcing until we figure out which species are key
add_source(rxn_id, "CPD0-2117[c]")
add_source(rxn_id, "ETF-Reduced[c]")
add_sink(rxn_id, "ETF-Oxidized[c]")
add_sink(rxn_id, "PALMITYL-COA[c]")
add_source_and_sink(rxn_id, "CPD0-2117[c]", "PALMITYL-COA[c]")

Growth rate with LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.5680 1/hr
Flux through LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.08543702919101856

Growth rate without LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.0000 1/hr
New reaction: CPD0-2117[c]_source
	CPD0-2117[c] <-- 		(np.float64(-0.08543702919101856), 0)
Growth rate with new reaction: 0.00 1/hr

Growth rate with LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.5680 1/hr
Flux through LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.08543702919101856

Growth rate without LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.: 0.0000 1/hr
New reaction: ETF-Reduced[c]_source
	ETF-Reduced[c] <-- 		(np.float64(-0.085437029

Looks like CPD0-2117[c] is required as a product and PALMITYL-COA[c] as a reactant. 

Looks like this reaction corresponds to 

Reaction 1.3.8.-

BioCyc ID: ACYLCOADEHYDROG-RXN

EC Number: 1.3.8 

https://biocyc.org/reaction?orgid=GCF_000011965&id=ACYLCOADEHYDROG-RXN

a 2,3,4-saturated fatty acyl CoA + oxidized electron-transfer flavoprotein + H+ → a (2E)-2-enoyl-CoA + reduced electron-transfer flavoprotein

However CPD0-2117 is not in the metacyc database.

In [253]:
m = model.metabolites.get_by_id("CPD0-2117[c]")
print(m.id, m.name, m.formula)

CPD0-2117[c] (2E)-hexadecenoyl-CoA C37H60N7O17P3S


CPD0-2117[c] is hdCoA!

The pathway from 2-4 are all linked and come together below

In [259]:
with model:
    model.reactions.get_by_id("FACOAE161").knock_out()
    model.reactions.get_by_id("RXN-10462").knock_out()
    
    # define relevant metabolites
    g3p = model.metabolites.get_by_id("GLYCEROL-3P[c]")
    hdcoa = model.metabolites.get_by_id("CPD0-2117[c]")
    coa = model.metabolites.get_by_id("CO-A[c]")
    acyl_sn_glycerol_3p = model.metabolites.get_by_id("ACYL-SN-GLYCEROL-3P[c]")
    l_phosphatidate = model.metabolites.get_by_id("L-PHOSPHATIDATE[c]")
    
    # define reaction 2.3.1.15
    rxn1 = cobra.Reaction("RXN_2.3.1.15")
    rxn1.add_metabolites({g3p: -1, hdcoa: -1, coa: 1, acyl_sn_glycerol_3p: 1})
    model.add_reactions([rxn1])
    
    # define reaction 2.3.1.51
    rxn2 = cobra.Reaction("RXN_2.3.1.51")
    rxn2.add_metabolites({acyl_sn_glycerol_3p: -1, hdcoa: -1, l_phosphatidate: 1, coa: 1})
    model.add_reactions([rxn2])
    
    # test model
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    sol = model.optimize()
    print(f"Growth rate with {sol.objective_value:.4f} 1/hr")
    
get_metabolite_reactions("POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c]")
    


Growth rate with 0.5687 1/hr
Consuming reactions:
	ENOYL-COA-DELTA-ISOM-RXN-POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24//CPD0-2117.53.: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] <=> CPD0-2117[c]
	FACOAE161: POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24[c] + WATER[c] <=> CO-A[c] + PROTON[c] + hdcea[c]
Producing reactions:



([<Reaction ENOYL-COA-DELTA-ISOM-RXN-POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24//CPD0-2117.53. at 0x1296a9e20>,
  <Reaction FACOAE161 at 0x1296b5f40>],
 [])

In [260]:
with model:
    model.reactions.get_by_id("FACOAE161").knock_out()
    model.reactions.get_by_id("RXN-10462").knock_out()
    model.reactions.get_by_id("ENOYL-COA-DELTA-ISOM-RXN-POLYMER-INST-CIS-DELTA3-ENOYL-COA-C12-H24//CPD0-2117.53.").knock_out()
    
    # define relevant metabolites
    g3p = model.metabolites.get_by_id("GLYCEROL-3P[c]")
    hdcoa = model.metabolites.get_by_id("CPD0-2117[c]")
    coa = model.metabolites.get_by_id("CO-A[c]")
    acyl_sn_glycerol_3p = model.metabolites.get_by_id("ACYL-SN-GLYCEROL-3P[c]")
    l_phosphatidate = model.metabolites.get_by_id("L-PHOSPHATIDATE[c]")
    
    # define reaction 2.3.1.15
    rxn1 = cobra.Reaction("RXN_2.3.1.15")
    rxn1.add_metabolites({g3p: -1, hdcoa: -1, coa: 1, acyl_sn_glycerol_3p: 1})
    model.add_reactions([rxn1])
    
    # define reaction 2.3.1.51
    rxn2 = cobra.Reaction("RXN_2.3.1.51")
    rxn2.add_metabolites({acyl_sn_glycerol_3p: -1, hdcoa: -1, l_phosphatidate: 1, coa: 1})
    model.add_reactions([rxn2])
    
    # test model
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    sol = model.optimize()
    print(f"Growth rate with {sol.objective_value:.4f} 1/hr")

Growth rate with 0.5687 1/hr


## 5. CDPDIGLYSYN-RXN-2

In [ ]:
LONG-CHAIN-ACYL-COA-DEHYDROGENASE-RXN-PALMITYL-COA/ETF-Oxidized/PROTON//CPD0-2117/ETF-Reduced.56.